In [53]:
# Лабораторная работа 3 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель рекуррентной сети с цепью нейросетевых моделей управляемых рекуррентных блоков 
# с логарифмической функцией активации (гиперболический арксинус) выходного сигнала на скрытом слое
#
# 24.12.24
# Данный файл содержит код построения графиков различных зависимостей

In [2]:
import numpy as np
from gru import (
    GRUModel,
    fibonacci_generator,
    one_by_n_generator,
    plus_one_minus_one_generator,
    squared_generator,
    half_generator,
    arithmetic_progression,
    create_sliding_window_batches,
    one_half_generator,
    sin_generator,    
    mape
)
from gru_with_adam import GRUAdam
import plotly.graph_objects as go
from math import pi

In [3]:
def scale_sequence(sequence):
    mean = np.mean(sequence)
    deviation = np.std(sequence)    
    scaled = (np.array(sequence) - mean) / deviation    
    return scaled, mean, deviation

def descale_sequence(scaled, mean, deviation):
    descaled = scaled * deviation + mean
    return descaled

def predict_next_n_elements(sequence, window_size, batch_size, hidden_size, n,
                            max_epochs, verbosity, lr, use_adam: bool = False):
    scaled, mean, deviation = scale_sequence(sequence)        
    # scaled = sequence  
    X, y = create_sliding_window_batches(scaled, window_size, batch_size, n)
    if not use_adam:
        model = GRUModel(window_size, hidden_size, n)
    else:
        model = GRUAdam(window_size, hidden_size, n)
    model.train(X, y, lr, max_epochs, verbosity=verbosity)
    train_pred, _, _ = model.forward(X)
    training_mape = mape(y, train_pred)           
    x_last = np.vstack(tuple(scaled[-window_size:] for _ in range(batch_size)))
    x_last = np.expand_dims(x_last, axis=0)    
    pred, _, _ = model.forward(x_last)
    descaled_pred = descale_sequence(pred[-1][-1], mean, deviation)
    # descaled_pred = pred[-1][-1]
    return training_mape, model, descaled_pred, mean, deviation

In [7]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
epochs = list(range(100, 2100, 100))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_epochs: list[int] = []

seq_length = 20
window_size = 5
batch_size = 8
hidden_size = 6
output_size = 3

verbosity = 100000
lr = 1e-2
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for max_epochs in epochs:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)
            

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 100/100, Loss: 0.351050
MAPE: 122.321234%
TRAINING FINISHED
Epoch 100/100, Loss: 0.263369
MAPE: 88.158012%
TRAINING FINISHED
Epoch 100/100, Loss: 0.297085
MAPE: 90.820335%
TRAINING FINISHED
Epoch 100/100, Loss: 0.277504
MAPE: 93.985166%
TRAINING FINISHED
Epoch 100/100, Loss: 0.318692
MAPE: 106.926804%
TRAINING FINISHED
Epoch 200/200, Loss: 0.218659
MAPE: 103.293494%
TRAINING FINISHED
Epoch 200/200, Loss: 0.236790
MAPE: 107.856342%
TRAINING FINISHED
Epoch 200/200, Loss: 0.173403
MAPE: 93.947970%
TRAINING FINISHED
Epoch 200/200, Loss: 0.214083
MAPE: 100.624506%
TRAINING FINISHED
Epoch 200/200, Loss: 0.183945
MAPE: 83.682312%
TRAINING FINISHED
Epoch 300/300, Loss: 0.166420
MAPE: 94.343167%
TRAINING FINISHED
Epoch 300/300, Loss: 0.158610
MAPE: 98.189489%
TRAINING FINISHED
Epoch 300/300, Loss: 0.170759
MAPE: 97.693217%
TRAINING FINISHED
Epoch 300/300, Loss: 0.097527
MAPE: 72.995486%
TRAINING FINISHED
Epoch 300/300, Loss: 0.191360
MAPE: 113.367783%
TRAINING FINISHED
E

In [8]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_epochs, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=epochs, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от количества итераций обучения',
    font=dict(size=20),
    xaxis=dict(title='Количество итераций обучения'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [28]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
hidden_sizes = list(range(2, 22, 2))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 20
window_size = 3
batch_size = 8
output_size = 3

verbosity = 100000
lr = 5e-3
max_epochs = 2000
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for size in hidden_sizes:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):
        all_sizes.append(size)        
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.042815
MAPE: 44.284933%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.044593
MAPE: 49.558001%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.177777
MAPE: 106.446637%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.009234
MAPE: 23.521857%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.034403
MAPE: 48.483827%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.044773
MAPE: 58.087223%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.060476
MAPE: 67.998126%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.048140
MAPE: 50.757371%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.007824
MAPE: 15.056716%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.012100
MAPE: 30.573573%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.035981
MAPE: 55.978345%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.041252
MAPE: 60.490237%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.041887
MAPE: 50.451606%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.010883
MAPE: 25.161759%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.043730
MAPE: 55.85

In [29]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=hidden_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>количества нейронов на скрытом слое',
    font=dict(size=20),
    xaxis=dict(title='Количество нейронов на скрытом слое'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [60]:
# MAPE relation with sliding window
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()


all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 40
window_sizes = list(range(2, seq_length // 2, 2))
hidden_size = 8
batch_size = 1
output_size = 3

verbosity = 100000
lr = 1e-2
max_epochs = 250
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for size in window_sizes:        

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):
        all_sizes.append(size)              
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)
        
        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 250/250, Loss: 0.048720
MAPE: 48.719204%
TRAINING FINISHED
Epoch 250/250, Loss: 0.035868
MAPE: 49.571113%
TRAINING FINISHED
Epoch 250/250, Loss: 0.091457
MAPE: 48.266458%
TRAINING FINISHED
Epoch 250/250, Loss: 0.048372
MAPE: 41.541267%
TRAINING FINISHED
Epoch 250/250, Loss: 0.111439
MAPE: 53.291036%
TRAINING FINISHED
Epoch 250/250, Loss: 0.022475
MAPE: 48.231422%
TRAINING FINISHED
Epoch 250/250, Loss: 0.036679
MAPE: 58.509946%
TRAINING FINISHED
Epoch 250/250, Loss: 0.052383
MAPE: 58.493765%
TRAINING FINISHED
Epoch 250/250, Loss: 0.037505
MAPE: 63.483636%
TRAINING FINISHED
Epoch 250/250, Loss: 0.058443
MAPE: 48.309907%
TRAINING FINISHED
Epoch 250/250, Loss: 0.029365
MAPE: 62.204312%
TRAINING FINISHED
Epoch 250/250, Loss: 0.023148
MAPE: 65.377842%
TRAINING FINISHED
Epoch 250/250, Loss: 0.035619
MAPE: 76.174040%
TRAINING FINISHED
Epoch 250/250, Loss: 0.030799
MAPE: 72.565365%
TRAINING FINISHED
Epoch 250/250, Loss: 0.040866
MAPE: 78.310398%
TRAINING FINISHED
Epoch 2

In [61]:
# Plotting results
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=window_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>размера скользящего окна',
    font=dict(size=20),
    xaxis=dict(title='Размер скользящего окна'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [30]:
# Настройки эксперимента
experiments_amount = 5
seq_length = 20
window_size = 3
batch_size = 8
hidden_size = 10
output_size = 3
max_epochs = 2_000
verbosity = 100000
lr = 1e-2

# Словарь с генераторами
sequence_generators = {
    "1, 1, 2, 3, 5, 8,...": lambda n: list(fibonacci_generator(n)),
    "1, 4, 9, 16,...": lambda n: list(squared_generator(n)),
    "1, 2, 3, 4, 5,...": lambda n: list(arithmetic_progression(n, a0=1, d=1)),    
    "1, 0.5 ,0.(3), 0.25, 0.2,...": lambda n: list(one_by_n_generator(n)),
    "1, -1, 1, -1,...": lambda n: list(plus_one_minus_one_generator(n)),
    "1, 0.5, 1, 0.5, 0.5, 1,...": lambda n: list(one_half_generator(n)),
}

all_mapes = []
mapes_mean = []
all_sequences = []

# Тренировка для каждой последовательности
for seq_name, seq_generator in sequence_generators.items():
    sequence = seq_generator(seq_length + output_size)
    print(sequence)   

    mapes_sum = 0
    for _ in range(experiments_amount):
        all_sequences.append(seq_name)
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)
        
        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)

    mapes_mean.append(mapes_sum / experiments_amount)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.149528
MAPE: 67.347948%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.109643
MAPE: 58.627691%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.144671
MAPE: 73.123521%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.163122
MAPE: 75.305979%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.140970
MAPE: 64.067471%
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.004528
MAPE: 140.956898%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.007529
MAPE: 127.546965%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.003928
MAPE: 168.069053%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.002112
MAPE: 134.817255%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.005674
MAPE: 139.169527%
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
TRAINING FINISHED
Epoch 2000/200

In [31]:
# Построение графика
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sequences, y=all_mapes, name="Все наблюдения", mode="markers")
)
fig.add_trace(
    go.Scatter(x=list(sequence_generators.keys()), y=mapes_mean, name="Средние значения", mode="markers")
)
fig.update_layout(
    title="Зависимость средней абсолютной ошибки<br>от вида последовательности",
    font=dict(size=20),
    xaxis=dict(title="Вид последовательности"),
    yaxis=dict(title="Средняя абсолютная ошибка"),
    width=1280,
    height=720
)
fig.show()